In [1]:
import pandas as pd 
from sklearn.model_selection import train_test_split 
import time
import numpy as np 
from sklearn.preprocessing import StandardScaler 
from sklearn.feature_selection import SelectKBest, chi2,RFE
from sklearn. feature_selection import RFE 
import pickle
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn. svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

In [2]:
def rfeFeature(indep_X,dep_Y,n):
        rfelist=[]
        from sklearn.linear_model import LinearRegression
        lin = LinearRegression()
        
        from sklearn.svm import SVR
        SVRl = SVR(kernel = 'linear')
        
        from sklearn.svm import SVR
        #SVRnl = SVR(kernel = 'rbf')
        
        from sklearn.tree import DecisionTreeRegressor
        dec = DecisionTreeRegressor(random_state = 0)
        
        from sklearn.ensemble import RandomForestRegressor
        rf = RandomForestRegressor(n_estimators = 10, random_state = 0)
        
        
        rfemodellist=[lin,SVRl,dec,rf] 
        for i in   rfemodellist:
            print(i)
            log_rfe = RFE(i, n_features_to_select= n)
            log_fit = log_rfe.fit(indep_X, dep_Y)
            log_rfe_feature=log_fit.transform(indep_X)
            rfelist.append(log_rfe_feature)
        return rfelist

In [3]:
def split_scalar(indep_X,dep_Y):
        X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size = 0.25, random_state = 0)
        #X_train, X_test, y_train, y_test = train_test_split(indep_X,dep_Y, test_size = 0.25, random_state = 0)
        
        #Feature Scaling
        #from sklearn.preprocessing import StandardScaler
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)    
        return X_train, X_test, y_train, y_test
    
def r2_prediction(regressor,X_test,y_test):
     y_pred = regressor.predict(X_test)
     from sklearn.metrics import r2_score
     r2=r2_score(y_test,y_pred)
     return r2
 
def Linear(X_train,y_train,X_test):       
        # Fitting K-NN to the Training set
        from sklearn.linear_model import LinearRegression
        regressor = LinearRegression()
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2   
    
def svm_linear(X_train,y_train,X_test):
                
        from sklearn.svm import SVR
        regressor = SVR(kernel = 'linear')
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2  
    
def svm_NL(X_train,y_train,X_test):
                
        from sklearn.svm import SVR
        regressor = SVR(kernel = 'rbf')
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2  
     

def Decision(X_train,y_train,X_test):
        
        # Fitting K-NN to the Training setC
        from sklearn.tree import DecisionTreeRegressor
        regressor = DecisionTreeRegressor(random_state = 0)
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2  
     

def random(X_train,y_train,X_test):       
        # Fitting K-NN to the Training set
        from sklearn.ensemble import RandomForestRegressor
        regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2 

In [4]:
def rfe_regression(acclin, accsvml, accsvmnl, accdes, accrf):
    dataframe=pd.DataFrame (index=['Linear','SVC','Random','Decision Tree' ], columns=['Linear', 'SVMl', 'SVMnl', 'Decision', 'Random'])
    for number,idex in enumerate (dataframe.index) :
         dataframe.loc[idex,'Linear']=acclin[number]
         dataframe.loc[idex,'SVMl']=accsvml [number]
         dataframe.loc [idex,'SVMnl']=accsvmnl [number]
         dataframe.loc[idex,'Decision']=accdes [number] 
         dataframe.loc[idex,'Random']=accrf [number ]
    return dataframe 

In [5]:
dataset1=pd. read_csv("prep.csv", index_col=None)
df2=dataset1
df2=pd.get_dummies(df2,drop_first=True)
df2=df2.drop('id', axis=1)
df2=df2.drop('classification_ckd\t', axis=1)
indep_X_scaled=df2.drop('classification_notckd', axis=1)
dep_Y=df2['classification_notckd']

In [6]:
reflist=rfeFeature( indep_X_scaled,dep_Y,5)          
acclin=[]
accsvml=[]
accsvmnl=[]
accdes=[]
accrf=[]

LinearRegression()
SVR(kernel='linear')
DecisionTreeRegressor(random_state=0)
RandomForestRegressor(n_estimators=10, random_state=0)


In [9]:
for i in reflist:
    X_train, X_test, y_train, y_test=split_scalar(i,dep_Y)
    r2_lin=Linear(X_train,y_train,X_test)
    acclin.append(r2_lin)
    
    r2_sl=svm_linear(X_train,y_train,X_test)    
    accsvml.append(r2_sl)
    
    r2_NL=svm_NL(X_train,y_train,X_test)
    accsvmnl.append(r2_NL)
    
    r2_d=Decision(X_train,y_train,X_test)
    accdes.append(r2_d)
    
    r2_r=random(X_train,y_train,X_test)
    accrf.append(r2_r)
result=rfe_regression(acclin, accsvml, accsvmnl, accdes, accrf)

In [10]:
result

,Linear,SVMl,SVMnl,Decision,Random
Linear,0.675004,0.679081,0.846138,0.87555,0.875878
SVC,0.673568,0.648708,0.880345,0.909877,0.909388
Random,0.726044,0.717864,0.936921,0.957555,0.971986
Decision Tree,0.720973,0.713735,0.951197,1.0,0.994329
